In [18]:
!pip install PyPDF2

distutils: /opt/anaconda3/include/python3.8/UNKNOWN
sysconfig: /opt/anaconda3/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /opt/anaconda3/include/python3.8/UNKNOWN
sysconfig: /opt/anaconda3/include/python3.8
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [19]:
from PyPDF2 import PdfReader

In [2]:
import re
import os, os.path
import fnmatch
import json
import collections
import glob
import json

In [21]:
def get_text_from_pdf(pdffile):
    # read text from pdf
    pdfreader = PdfReader(pdffile)
    raw_text = ''
    for i, page in enumerate(pdfreader.pages):
        content = page.extract_text()
        if content:
            raw_text += content
    return raw_text

In [34]:
rootdir = "/Users/agniksaha/Documents/Resume_classification/data/"
df_dict = {}

keys = []
texts = []

for path in glob.glob(f'{rootdir}/*/**/', recursive=True):
    key = str(path.split('/')[-2])
    files = [f for f in os.listdir(path)]
    for f in files:
        dir_path = path+f
        texts.append(get_text_from_pdf(dir_path))   
        keys.append(key)


In [35]:
import pandas as pd

df = pd.DataFrame()
df['label'] = keys
df['text'] = texts

df.to_csv("resumes_from_pdf.csv")

In [36]:
len(df)

2484

In [37]:
def convert_to_lower(text):
    return text.lower()

def remove_numbers(text):
    text = re.sub(r'd+' , '', text)
    return text

def remove_white_space(text):
    text = text.strip()
    return text

def preprocess_text(text):
    text = remove_numbers(text)
    text = convert_to_lower(text)
    text = remove_white_space(text)
    return text

from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT
def get_keyword_bert(docs):
    try:
        vectorizer = KeyphraseCountVectorizer()
        kw_model = KeyBERT()
        onewords = kw_model.extract_keywords(docs=docs, vectorizer=vectorizer,top_n=5,stop_words='english')
        return onewords
    except:
        pass        
    
# !pip install multi_rake
from multi_rake import Rake

def get_keywords_rake(full_text):
    try:
        rake = Rake()
        keywords = rake.apply(full_text)
        return keywords
    except:
        pass
    
    
df['keywords_bert'] = df.text.apply(lambda x: get_keyword_bert(x))
df['keywords_rake'] = df.text.apply(lambda x: get_keywords_rake(x))

df.to_csv("resumes_from_pdf_2.csv")


In [38]:
df

,label,text,keywords_bert,keywords_rake
0,AGRICULTURE,ADULT EDUCATION INSTRUCTOR\nSummary\nSeasoned ...,"[(adult education instructor, 0.6756), (adult ...","[(covina high school, 9.0), (bloomington high ..."
1,AGRICULTURE,FINANCIAL SALES CONSULTANT\nProfessional Summa...,"[(financial sales consultant, 0.662), (client ...","[(press kit construction, 9.0), (youth activis..."
2,AGRICULTURE,EXTENSION METHODOLOGIST\nProfile\nSelf-motivat...,"[(extension methodologist, 0.645), (agricultur...","[(yielding good results, 9.0), (giving excelle..."
3,AGRICULTURE,RESEARCH SCIENTIST\nSummary\nHighly motivated ...,"[(root nodule associated bacteria, 0.5252), (y...","[(characterize epigenetic modifications, 9.0),..."
4,AGRICULTURE,"FRONT DESK CLERK (FEE BASIS, JOHN D DINGELL VA...","[(front desk clerk, 0.5373), (customer service...","[(earned high marks, 9.0), (work process flow,..."
...,...,...,...,...
2479,AUTOMOBILE,GENERAL LIABILITY CLAIM REPRESENTATIVE\nSummar...,"[(automobile liability claims representative, ...","[(successfully indentifying resources, 9.0), (..."
2480,AUTOMOBILE,AUTOMOBILE TRANSPORTER\nProfessional Summary\n...,"[(automobile transporter, 0.6479), (car transp...","[(clean driver's license, 9.0), (exceptional l..."
2481,AUTOMOBILE,"Highlights\nProg. Languages: \nC (5+ yrs), Pyt...","[(programming, 0.4858), (statistics softwares,...","[(makerbot 3d print, 9.0), (undergraduate thes..."
2482,AUTOMOBILE,CUSTOMER RELATIONS SPECIALIST\nSummary\nTo obt...,"[(customer relations specialist, 0.6615), (cus...","[(starbucks store managers, 9.0), (recommends ..."


In [40]:
df['label'].value_counts()

INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
ACCOUNTANT                118
ADVOCATE                  118
FINANCE                   118
ENGINEERING               118
CHEF                      118
AVIATION                  117
FITNESS                   117
SALES                     116
HEALTHCARE                115
BANKING                   115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: label, dtype: int64